In [26]:
import pandas as pd
import quandl as qd
import math
import numpy as np
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression


df =qd.get("WIKI/GOOGL")

print(df.head())

              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2004-08-19  100.01  104.06   95.96  100.335  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.310  22834300.0          0.0   
2004-08-23  110.76  113.48  109.05  109.400  18256100.0          0.0   
2004-08-24  111.24  111.60  103.57  104.870  15247300.0          0.0   
2004-08-25  104.76  108.00  103.88  106.000   9188600.0          0.0   

            Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  \
Date                                                                   
2004-08-19          1.0  50.159839  52.191109  48.128568   50.322842   
2004-08-20          1.0  50.661387  54.708881  50.405597   54.322689   
2004-08-23          1.0  55.551482  56.915693  54.693835   54.869377   
2004-08-24          1.0  55.792225  55.972783  51.945350   52.597363   
2004-08-25          1.0  52.542193  54.167209  52.100830   53.1

Como pode ser visto o stock market apresenta 12 features. Nesse ponto é imteressante se perguntar se todas elas são necessárias para a previsão, para checar isso é possível fazer alguns teste como são demonstrado abaixo


In [27]:
df = df[["Adj. Open","Adj. High","Adj. Low","Adj. Close","Adj. Volume"]]
df["Hl_porcent"] = (df["Adj. High"]- df["Adj. Close"])/df["Adj. Close"]*100
df["porcent_change"] = (df["Adj. Close"]- df["Adj. Open"])/df["Adj. Open"]*100
df = df[["Adj. Close","Hl_porcent","porcent_change","Adj. Volume"]]

print(df.head())

            Adj. Close  Hl_porcent  porcent_change  Adj. Volume
Date                                                           
2004-08-19   50.322842    3.712563        0.324968   44659000.0
2004-08-20   54.322689    0.710922        7.227007   22834300.0
2004-08-23   54.869377    3.729433       -1.227880   18256100.0
2004-08-24   52.597363    6.417469       -5.726357   15247300.0
2004-08-25   53.164113    1.886792        1.183658    9188600.0


In [28]:
forecast_col = "Adj. Close" #Label para o valor do preço
df.fillna(-99999, inplace = True) 
#not avaiable,pandas não trabalham com valores que não são números, então é interessante subistituir por um outlier
#para não perder uma coluna

forecast_out = int(math.ceil(0.001*len(df)))
#math arredonda o valor, mas retorna como float. O interior do ceil é o número de dias que iremos prever

df["label"] = df[forecast_col].shift(-forecast_out)
#permite mudar o nome da label, enquanto shift vai mover para cima o número de colunas para, assim cada linha será o valor de adj close
df.dropna(inplace = True)
print(df.tail())

            Adj. Close  Hl_porcent  porcent_change  Adj. Volume    label
Date                                                                    
2018-03-15     1150.61    1.033365        0.090469    1623868.0  1094.00
2018-03-16     1134.42    1.973696       -1.811572    2654602.0  1053.15
2018-03-19     1100.07    1.754434       -1.582630    3076349.0  1026.55
2018-03-20     1095.80    0.889761       -0.236708    2709310.0  1054.09
2018-03-21     1094.00    1.343693        0.130884    1990515.0  1006.94


In [33]:
X = np.array(df.drop(['label'],1))#retorna o df

X = preprocessing.scale(X)#Lembrar de escalonar os valores

X_lately = X[-forecast_out:]



y = np.array(df["label"])
df.dropna(inplace=True)


X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y, test_size = 0.2)


ValueError: Found input variables with inconsistent numbers of samples: [3416, 3420]

In [ ]:
clf = LinearRegression(n_jobs=-1)#n_jobs é o número de treads que podem ser rodadas paralelamente(-1 excolhe pro n° maximo que a sua cpu consegue)
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print("Estamos medindo:",forecast_out,"dias de antecedência com", accuracy,"de acurácia")

Utilizando outro algoritmo

In [ ]:
clf_svm = svm.SVR()
clf_svm.fit(X_train, y_train)
accuracy = clf_svm.score(X_test, y_test)
print("Estamos medindo:",forecast_out,"dias de antecedência com", accuracy,"de acurácia")

In [ ]:
clf_svm = svm.SVR(kernel = "poly")
clf_svm.fit(X_train, y_train)
accuracy = clf_svm.score(X_test, y_test)
print("Estamos medindo:",forecast_out,"dias de antecedência com", accuracy,"de acurácia")